In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
import requests
import pandas as pd
import json
import talib
import numpy as np
import time
import datetime
import yfinance as yf
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

warnings.filterwarnings('ignore')



# Load API keys - search in multiple locations
def find_secrets():
    """Find secrets.json in current dir, parent dir, or submodules dir."""
    possible_paths = [
        "../secrets.json",  # Parent directory (markets/)
    ]
    for path in possible_paths:
        if os.path.exists(path):
            print(f"✓ Found secrets at: {path}")
            return path
    raise FileNotFoundError(
        "secrets.json not found. Searched locations:\n" + "\n".join(f"  - {p}" for p in possible_paths)
    )

SECRET_FP = find_secrets()
with open(SECRET_FP, 'r') as file:
    secrets = json.load(file)
API_KEY = secrets['eodhd_api_key']

# Import from package (works if installed with pip install -e .)
try:
    from markets.submodules import Chain, ChainHistory, Security, Index
    from markets.submodules.fa import FundamentalAnalysis
    from markets.submodules import plot
    from markets.submodules.plot import ChartStyle
    print("✓ Using installed markets package")
except ImportError:
    print("⚠ Package not installed, using local imports")
    
    # Try to find the submodules directory
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    
    # Check for markets/submodules (if we're in markets/implementations)
    submodules_path = os.path.join(parent_dir, 'submodules')
    
    if os.path.exists(submodules_path):
        sys.path.insert(0, submodules_path)
        print(f"  Added to path: {submodules_path}")
    
    from eodhd import Chain, ChainHistory, Security, Index, News
    import plot
    from plot import ChartStyle

cs = ChartStyle(figsize=(20,8),nyears=1, date_fmt="%Y-%m", grid=False)

✓ Found secrets at: ../secrets.json
⚠ Package not installed, using local imports
  Added to path: c:\Users\msands\OneDrive\Documents\code\markets\submodules
✓ News class created
✓ News class created


In [2]:
News().get_ticker_news(ticker='AAPL', limit=10, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment
0,2025-12-09 01:00:29+00:00,Investor Quits 9-to-5 Job After Reaching $14K ...,Concerns about stretched tech valuations and a...,https://finance.yahoo.com/news/investor-quits-...,"[AAPL.US, APP.US, AVGO.US, BLOX.US, CIFR.US, E...","[DIVIDENDS, ETFS, HIGH-YIELD, INCOME-STRATEGY,...","{'polarity': 0.996, 'neg': 0.024, 'neu': 0.866..."
1,2025-12-09 00:31:07+00:00,Jeff Bezos Once Said The Universe Wants You To...,Amazon.com Inc.(NASDAQ:AMZN) founder Jeff Bezo...,https://finance.yahoo.com/news/jeff-bezos-once...,"[AAPL.US, AMZN.US, TSLA.US]","[RETAIL, SHARE-BUYBACK, SHAREHOLDER, TECH]","{'polarity': 0.988, 'neg': 0.032, 'neu': 0.865..."
2,2025-12-09 00:24:00+00:00,Should You Buy MP Materials Stock While It's U...,Key Points\n\nMP's rare-earth metal mine in Ca...,https://www.nasdaq.com/articles/should-you-buy...,"[AAPL.US, MP.US]","[DEFENSE, MANUFACTURING, MARKETS, MINING, RARE...","{'polarity': 0.989, 'neg': 0.049, 'neu': 0.843..."
3,2025-12-08 23:36:13+00:00,Apple Faces Leadership Turmoil As AI Setbacks ...,(RTTNews) - Apple (AAPL) is undergoing an unus...,https://www.nasdaq.com/articles/apple-faces-le...,[AAPL.US],"[ARTIFICIAL-INTELLIGENCE, HARDWARE, LEADERSHIP...","{'polarity': 0.09, 'neg': 0.047, 'neu': 0.909,..."
4,2025-12-08 20:57:53+00:00,Apple's chip chief reportedly 'not leaving': A...,Amid a flurry of executive departures from App...,https://finance.yahoo.com/video/apples-chip-ch...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...","[AI, CHIP-DESIGN, EXECUTIVE-DEPARTURES, TECH]","{'polarity': 0.998, 'neg': 0.014, 'neu': 0.887..."
5,2025-12-08 20:39:37+00:00,Nvidia stock rises on report Trump administrat...,Shares of Nvidia (NVDA) rose on Monday followi...,https://finance.yahoo.com/news/nvidia-stock-ri...,"[1AAPL.MI, 1AMZN.MI, 1FB.MI, 1GOOG.MI, 1GOOGL....","[AI, CHIPS, NATIONAL-SECURITY, QUARTERLY EARNI...","{'polarity': 0.894, 'neg': 0.056, 'neu': 0.86,..."
6,2025-12-08 20:39:37+00:00,Trump approves Nvidia H200 chips for sale to C...,President Trump on Monday announced that he wi...,https://finance.yahoo.com/news/trump-approves-...,"[AAPL.US, AMZN.US, GOOG.US, GOOGL.US, META.US,...","[AI, EARNINGS, NATIONAL-SECURITY, QUARTERLY EA...","{'polarity': 0.801, 'neg': 0.057, 'neu': 0.868..."
7,2025-12-08 19:50:00+00:00,Meta Platforms Stock Jumps on Metaverse Spendi...,Key Points\n\nMeta Platforms has lost a stunni...,https://www.nasdaq.com/articles/meta-platforms...,"[AAPL.US, AMZN.US, GOOGL.US, META.US, MSFT.US,...","[AI, BALANCE SHEET, FREE CASH FLOW, GROWTH-STO...","{'polarity': 0.998, 'neg': 0.049, 'neu': 0.825..."
8,2025-12-08 19:37:38+00:00,"Tech Voices: Trump's AI order, Nvidia-China, A...","[Nvidia Corporation building in Taipei, Taiwan...",https://seekingalpha.com/news/4529438-tech-voi...,"[AAPL.US, AMD.US, NVDA.US]","[REGULATION, SEMICONDUCTORS, TECH]","{'polarity': 0.885, 'neg': 0.063, 'neu': 0.848..."
9,2025-12-08 19:23:43+00:00,Apple’s Chips Chief Says He Isn’t Leaving,Apple appears to be keeping hold of a key exec...,https://finance.yahoo.com/m/163a18fc-852b-3403...,"[1AAPL.MI, AAPL.BA, AAPL.MX, AAPL.US, AAPL34.S...",[],"{'polarity': -0.625, 'neg': 0.074, 'neu': 0.92..."


In [3]:
News().get_latest_news( limit=10, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment
0,2025-12-09 04:30:32+00:00,Paramount Skydance Corporation declares $0.05 ...,\t* Paramount Skydance Corporation (PSKY [htt...,https://seekingalpha.com/news/4529520-paramoun...,[PSKY.US],"[DIVIDEND STOCKS NEWS, DIVIDENDS, MEDIA]","{'polarity': 0.687, 'neg': 0.051, 'neu': 0.854..."
1,2025-12-09 04:27:00+00:00,Responding to Change through Reform Pursuing D...,"GUANGZHOU, China, Dec. 08, 2025 (GLOBE NEWSW...",https://www.globenewswire.com/news-release/202...,[],"[CALENDAR OF EVENTS, ECONOMIC GOVERNANCE REFOR...","{'polarity': 1, 'neg': 0.051, 'neu': 0.796, 'p..."
2,2025-12-09 04:26:58+00:00,"Australia holds interest rate steady, flags in...",[Australia Flag]\nhusayno\n\nThe Reserve Bank ...,https://seekingalpha.com/news/4529518-australi...,"[AUD:USD.US, EWA.US, FLAU.US, FXA.US, USD:AUD.US]","[INFLATION, MONETARY-POLICY, RATES, RATINGS]","{'polarity': 0.957, 'neg': 0.048, 'neu': 0.845..."
3,2025-12-09 04:19:00+00:00,"Nantong, Jiangsu: E-commerce Training Helps En...","NANTONG, China, Dec. 08, 2025 (GLOBE NEWSWIR...",https://www.globenewswire.com/news-release/202...,[],"[CROSS-BORDER OPERATIONS, CROSS-BORDER-TRADE, ...","{'polarity': 0.949, 'neg': 0, 'neu': 0.905, 'p..."
4,2025-12-09 03:53:27+00:00,AUD/JPY weakens below 103.50 as RBA leaves int...,The AUD/JPY cross loses momentum to near 103.2...,https://www.fxstreet.com/news/aud-jpy-weakens-...,"[AUD-JPY.US, AUD-USD.US, AUDUSD.FOREX, JPYUSD....",[RATES],"{'polarity': -0.128, 'neg': 0.12, 'neu': 0.8, ..."
5,2025-12-09 03:51:00+00:00,eSignGlobal Forms Strategic Alliance with Lion...,"HANGZHOU, China, Dec. 9, 2025 /PRNewswire/ -- ...",https://finance.yahoo.com/news/esignglobal-for...,"[OCBA.F, OCBB.F, OVCHF.US, OVCHY.US]","[ASEAN, CROSS-BORDER-TRADE, DIGITAL-ECONOMY, E...","{'polarity': 0.994, 'neg': 0.012, 'neu': 0.862..."
6,2025-12-09 03:43:45+00:00,Ares Management To Replace Kellanova In S&amp;...,"(RTTNews) - S&P Dow Jones Indices, a division ...",https://www.nasdaq.com/articles/ares-managemen...,"[ARES.US, CRGY.US, HSII.US, K.US, SEZL.US, SPG...","[ENERGY, INDICES, M-A, MARKETS]","{'polarity': 0.935, 'neg': 0.027, 'neu': 0.855..."
7,2025-12-09 03:38:27+00:00,AUD/NZD slides to near 1.1440 as RBA holds OCR...,The AUD/NZD pair falls sharply to near 1.1440 ...,https://www.fxstreet.com/news/aud-nzd-slides-t...,"[AUD-NZD.US, AUDUSD.FOREX, NZDUSD.FOREX]","[INFLATION, RATES]","{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"
8,2025-12-09 03:26:00+00:00,Daedalus Special Acquisition Corp. Announces t...,"London, United Kingdom, Dec. 08, 2025 (GLOBE...",https://www.globenewswire.com/news-release/202...,"[DSAC.US, DSACU.US, DSACW.US]","[AI, INITIAL PUBLIC OFFERING, IPO, NASDAQ, PRE...","{'polarity': 0.993, 'neg': 0.014, 'neu': 0.889..."
9,2025-12-09 03:21:21+00:00,BHP Signs US$2 Bln Agreement With GIP For WAIO...,(RTTNews) - BHP announced that it has entered ...,https://www.nasdaq.com/articles/bhp-signs-us-2...,"[BHP.US, BLK.US]","[INFRASTRUCTURE, IRON-ORE, M-A, MARKETS]","{'polarity': 0.977, 'neg': 0.006, 'neu': 0.86,..."


In [4]:
News().get_multiple_tickers_news( tickers = ['BAC', 'JPM' , 'C'], limit=4, from_date = '2025-12-01')

,date,title,content,link,symbols,tags,sentiment,primary_ticker
0,2025-12-08 22:18:00+00:00,"AI Policy, Chips, &amp; China: What Investors ...","Outside of the Federal Reserve and trade, noth...",https://finance.yahoo.com/news/ai-policy-chips...,"[BIDU.US, C.MX, C.US, CTGP34.SA, NVDA.US, TRVC...","[AI, IPO, PRICE TARGET, REGULATION, SEMICONDUC...","{'polarity': 0.993, 'neg': 0.024, 'neu': 0.876...",C
1,2025-12-08 22:03:34+00:00,"Banks have a busy week: Fed decision, Goldman ...",It's an important week for banks and the finan...,https://finance.yahoo.com/video/banks-busy-wee...,"[BAC-PB.US, BAC-PE.US, BAC-PK.US, BAC-PL.US, B...","[BANKS, CRYPTO, FINANCIALS, REGULATION]","{'polarity': 0.979, 'neg': 0.006, 'neu': 0.919...",BAC
2,2025-12-08 21:38:04+00:00,"Stocks to Watch Monday Recap: IBM, Boeing, Bro...",↗️ Boeing (BA): The plane maker said Monday it...,https://finance.yahoo.com/m/a20f2217-8d50-32dd...,"[1MU.MI, 1YD.BE, 1YD.DU, 1YD.HA, 1YD.HM, 1YD.M...","[INDUSTRIALS, M-A, TECH]","{'polarity': 0.178, 'neg': 0, 'neu': 0.964, 'p...",JPM
3,2025-12-08 20:51:46+00:00,Sector Update: Financial Stocks Decline Late A...,Financial stocks were decreasing in late Monda...,https://finance.yahoo.com/news/sector-financia...,"[BLK.US, BRK.A, BTC.CC, GSPC.INDX, JPM.US, XLF...",[],"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}",JPM
4,2025-12-08 20:51:06+00:00,"Stock market today: Dow, S&amp;P 500, Nasdaq f...",US stocks fell on Monday as Wall Street awaits...,https://finance.yahoo.com/news/live/stock-mark...,"[1NVDA.MI, 700.US, BRK.US, CFLT.US, DJI.INDX, ...","[EARNINGS, FED, M-A, RATE-CUT, SPLIT, TECH]","{'polarity': 0.999, 'neg': 0.051, 'neu': 0.851...",JPM
5,2025-12-08 19:49:33+00:00,Big bank CEOs to meet with senators to discuss...,[Business Signage]\nBruce Bennett/Getty Images...,https://seekingalpha.com/news/4529437-big-bank...,"[BAC.US, C.US, WFC.US]","[BANKS, CRYPTO, CRYPTOCURRENCY , DIGITAL-ASSET...","{'polarity': 0.993, 'neg': 0.006, 'neu': 0.837...",BAC
6,2025-12-08 17:30:00+00:00,Umicore - Transparency notification by Bank of...,Umicore\n\nTransparency notification by Bank o...,https://finance.yahoo.com/news/umicore-transpa...,"[BAC-PB.US, BAC-PE.US, BAC-PK.US, BAC-PL.US, B...",[],"{'polarity': 0.981, 'neg': 0.011, 'neu': 0.899...",BAC
7,2025-12-08 17:25:00+00:00,"UBS Group AG Plans 10,000 Job Cuts: Will it Bo...","UBS Group AG UBS is preparing to cut up to 10,...",https://finance.yahoo.com/news/ubs-group-ag-pl...,"[BLK.US, C.US, UBS.US, UBSG.SW, UBSG34.SA, UBS...","[BANKS, FINANCIALS, FOURTH QUARTER, M-A, RESTR...","{'polarity': 0.997, 'neg': 0.023, 'neu': 0.837...",C
8,2025-12-08 17:12:05+00:00,Paramount Skydance ups the stakes for Warner B...,"UPDATED 11:55ET to include Affinity Partners, ...",https://seekingalpha.com/news/4529397-paramoun...,"[AMZN.US, APO.US, BAC.US, C.US, CNK.US, DIS.US...","[M-A, MEDIA, STREAMING]","{'polarity': 0.996, 'neg': 0.037, 'neu': 0.822...",BAC


In [5]:
News().search_news(keywords='Fed', from_date = '2025-11-01', to_date = datetime.today().strftime('%Y-%m-%d'))

,date,title,content,link,symbols,tags,sentiment
0,2025-12-08 23:59:46+00:00,"Singapore Bourse May Give Up Support At 4,500 ...",(RTTNews) - The Singapore stock market has tra...,https://www.nasdaq.com/articles/singapore-bour...,[],[],None
1,2025-12-08 23:59:12+00:00,D-Wave Quantum's Stock Price Crashed Nearly 40...,Key Points\n\nD-Wave's stock suffered one of t...,https://www.nasdaq.com/articles/d-wave-quantum...,"[QBTS.US, RGTI.US]","[GOVERNMENT-INVESTMENT, MARKET-VOLATILITY, MAR...","{'polarity': 0.991, 'neg': 0.065, 'neu': 0.824..."
2,2025-12-08 23:58:07+00:00,Stocks Retreat as Bond Yields Rise,The S&P 500 Index ($SPX) (SPY) on Monday close...,https://www.nasdaq.com/articles/stocks-retreat...,"[AMD.US, APD.US, ASML.US, AVGO.US, AZO.US, BSX...","[EARNINGS, FOMC, INTEREST-RATES, M-A, PRICE TA...","{'polarity': 0.994, 'neg': 0.042, 'neu': 0.879..."
3,2025-12-08 23:44:50+00:00,US stocks end lower as investors wait for Fed ...,STORY: Wall Street's main indexes closed lower...,https://finance.yahoo.com/video/us-stocks-end-...,"[CFLT.US, CVNA.US, DJI.INDX, GSPC.INDX, IBM.US...","[FED, M-A, RATES, TECH, UTILITIES]","{'polarity': 0.948, 'neg': 0.064, 'neu': 0.821..."
4,2025-12-08 23:30:39+00:00,"Stock market today: Dow, S&amp;P 500, Nasdaq f...",US stock futures edged higher Monday night aft...,https://finance.yahoo.com/news/live/stock-mark...,"[AVGO.US, COST.US, DJI.INDX, GSPC.INDX, IXIC.I...","[AI, EARNINGS, RATE-CUT, TECH, TRADE-POLICY]","{'polarity': 0.838, 'neg': 0.044, 'neu': 0.879..."
5,2025-12-08 23:29:46+00:00,Losing Streak May Continue For Malaysia Stock ...,(RTTNews) - The Malaysia stock market has fini...,https://www.nasdaq.com/articles/losing-streak-...,[],[],None
6,2025-12-08 23:15:13+00:00,"Tech sector streak, Santa Claus indicator: Mar...",Yahoo Finance Markets and Data Editor Jared Bl...,https://finance.yahoo.com/video/tech-sector-st...,"[GSPC.INDX, SOXX.US, XLK.US]","[SANTA-CLAUS-RALLY, TECH, YIELDS]","{'polarity': 0.992, 'neg': 0.037, 'neu': 0.879..."
7,2025-12-08 23:11:43+00:00,Does Thyssenkrupp’s 138% Surge in 2025 Still L...,Wondering if thyssenkrupp is still a bargain a...,https://finance.yahoo.com/news/does-thyssenkru...,"[1TKA.MI, TKA.BE, TKA.DU, TKA.F, TKA.HM, TKA.M...","[FREE CASH FLOW, FUTURE CASH FLOWS, INTRINSIC ...","{'polarity': 0.999, 'neg': 0.048, 'neu': 0.788..."
8,2025-12-08 23:05:10+00:00,Asian Penny Stocks To Watch In December 2025,As global markets anticipate the Federal Reser...,https://finance.yahoo.com/news/asian-penny-sto...,"[2177.HK, 2482.HK, B8O.F, BS6.SG, BYD-R.BK, BY...","[BALANCE SHEET, FINANCIALS, FREE CASH FLOW, GR...","{'polarity': 0.981, 'neg': 0.031, 'neu': 0.892..."
9,2025-12-08 23:03:00+00:00,"ROSEN, NATIONAL TRIAL ATTORNEYS, Encourages Wi...","NEW YORK, Dec. 08, 2025 (GLOBE NEWSWIRE) --\...",https://www.globenewswire.com/news-release/202...,[],"[CLASS ACTION, INVESTOR-RIGHTS, LAW & LEGAL IS...","{'polarity': 0.992, 'neg': 0.055, 'neu': 0.817..."
